In [ ]:
import os
import re
import json
from tqdm import tqdm
from utils.call_gpt import call_gpt, call_gpt_async, read_table_file

# 1. 提取关键词

## 1.1 prompt模板

In [2]:
prompt_get_keywords = """
您作为法律语义解析专家，请严格遵循以下规则从法律文本中提取规范关键字：

【术语定义】
1. 最小原子性 - 指不可拆分的完整法律概念（如"持股5%以上股东"不可拆为"股东"）
2. 形式唯一性 - 必须完全匹配法律条文中的原始表述形式
3. 复合结构(不强求复合结构, 但如果有结构则需要保留) - 包含以下特征：
   - 含数字量化（如"5%以上"）
   - 含主体关系链（如"一致行动人"的三级归属结构）
   - 含交易方式限定（如"竞价交易"）
   - 含特定机构称谓（如"本所"）

【提取规范】
✅ 应提取：
- 具有法律效力的主体身份表述
- 在本法律下特定的行为的专业表述
- 如果没有关键字就不需要返回关键字

❌ 不应提取：
- 动词短语/形容词短语
- 非固定搭配的普通词汇
- 语义相同但表述不同的同义词
- 超过20字的长句. 一般来说关键字不会这么长
- 特定法律的名称. 这种属于外部引用, 而非本法规内部的关键字
- 单独的时间限制, 例如"最近三十六个月内"
- 出现少于二次的词语. 我们希望找到的是频繁出现的关键术语, 如果这个术语出现很少, 没有提取的必要. 
- “（如有）”这种标签
- 过于常见的词汇, 例如"上市公司", "律师事务所"
- 缺乏实际意义的词汇, 例如"持续发展"

【参考范例】
优秀案例：
["控股股东", "持股5%以上股东", "本所"]
需避免案例：
["股东", "所持股份比例在5%以上的股东", "证券交易所"]

【输出要求】
1. 严格保持法条原始字词组合
2. 按出现顺序排列
3. Python列表格式，元素用双引号包裹
4. 用<KEYWORDS></KEYWORDS>标签封装

下面是需要你处理的法律条文：
{Law_articles}
"""


In [3]:

# content, reasoning_content, api_usage = call_gpt(
#     prompt=prompt_get_keywords.format(Law_articles=batches[1]),
#     api_key = "35684824-1776-48b6-94fd-96c2e99d0724",
#     base_url = "https://ark.cn-beijing.volces.com/api/v3",
#     model = "ep-20250217153824-9xcbx",  # DeepSeek-R1
# )
# reasoning_content

In [4]:
# print(content)

## 1.2 异步获取keywords结果

In [ ]:
import pandas as pd
import asyncio
import os


async def get_keywords_response_async(
    file_name: str,
    batch_size: int = 20,
    max_concurrency: int = 5  # 最大并发数
):
    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir = r"law_to_MEU/law_articles_csv"
    output_dir = r"law_to_MEU/law_articles_keywords/raw_response"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    df = pd.read_csv(input_path, encoding='utf-8-sig')
    law_articles = df['law_article'].tolist()

    batches = [law_articles[i:i+batch_size] 
              for i in range(0, len(law_articles), batch_size)]
    
    tasks = []
    for batch in batches:
        law_text = "\n".join(batch)
        prompt = prompt_get_keywords.format(Law_articles=law_text)
        task = call_gpt_async(
            prompt=prompt,
            api_key="35684824-1776-48b6-94fd-96c2e99d0724",
            base_url="https://ark.cn-beijing.volces.com/api/v3",
            model="ep-20250217153824-9xcbx",
            temperature=0.6,
        )
        tasks.append(task)

    # 分组执行控制并发
    all_results = []
    for i in range(0, len(tasks), max_concurrency):
        batch_tasks = tasks[i:i + max_concurrency]
        batch_results = await asyncio.gather(*batch_tasks)
        all_results.extend(batch_results)

    contents = []
    reasonings = []
    usages = []
    
    for content, reasoning, usage in all_results:
        contents.append(content)
        reasonings.append(reasoning)
        usages.append(usage)

    combined_df = pd.DataFrame({
        'Content': contents,
        'Reasoning': reasonings,
        'Usage': usages
    })
    
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    return contents, reasonings, usages

In [6]:
# file_name = "北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv"
# await get_keywords_response_async(file_name)

## 1.3 解析并保存keywords

In [7]:
import pandas as pd
import re
import ast
import os

def get_keywords(input_file):

    if input_file.endswith(".csv"):
        pass
    else:
        input_file += ".csv"

    # 构建路径
    input_path = os.path.join("law_to_MEU", "law_articles_keywords", "raw_response", input_file)
    output_dir = os.path.join("law_to_MEU", "law_articles_keywords")
    output_path = os.path.join(output_dir, input_file)
    
    # 读取原始文件
    df = pd.read_csv(input_path, encoding='utf-8-sig')
    
    all_keywords = []
    
    for content in df['Content']:
        # 替换错误的闭合标签
        content = content.replace("<\KEYWORDS>", "</KEYWORDS>")
        
        # 提取关键词内容
        match = re.search(r'<KEYWORDS>(.*?)</KEYWORDS>', content, re.DOTALL)
        if match:
            keywords_str = match.group(1).strip()
            try:
                # 转换字符串为列表
                keywords_list = ast.literal_eval(keywords_str)
                if isinstance(keywords_list, list):
                    all_keywords.extend(keywords_list)
            except (SyntaxError, ValueError):
                print(f"解析失败的内容：{keywords_str}")
                continue
    
    # 去重并创建DataFrame
    unique_keywords = pd.DataFrame({'keywords': list(set(all_keywords))})
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存结果
    unique_keywords.to_csv(output_path, index=False, encoding='utf-8-sig')
    return unique_keywords

In [8]:

# file_name = "北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv"
# result_df = get_keywords(file_name)
# result_df

# 2. 提取全局定义

In [9]:
prompt_extract_definitions = """
法律条文全局定义提取器
识别法律文本中通过统称、列举、解释等方式建立的全局性定义，
构建术语与完整定义的映射关系。

【定义特征识别】
典型模式：
1. 统称："...（以下统称XX）" 
2. 简称："...（以下简称XX）"
3. 解释："XX指..." 
4. 复合："XX包括...，且需满足...条件"
5. 所称: "本指引所称的xxx是xxx"

【任务要求】
1. 捕获定义的完整语义边界
2. 保留原文的限定词与逻辑关系
3. 处理包含条件句、例外条款的复杂定义
4. 如果没有全局定义就不需要返回全局定义, 不要杜撰全局定义

EXAMPLES
>>> input：
"上市公司控股股东、实际控制人和持有5%以上股份的股东（以下统称大股东）..."
<<< output：
{{"大股东": "上市公司控股股东、实际控制人、持有5%以上股份的股东的统称"}}

>>> input：
"本细则所称异常交易，指连续30个交易日成交量超过市场均值200%且被本所警示的交易行为"
<<< output：
{{"异常交易": "连续30个交易日成交量超过市场均值200%且被本所警示的交易行为"}}

>>> input：
"重点排污单位(含年排放量超5000吨的工业企业、市级以上污水处理厂)"
<<< output：
{{"重点排污单位": "包含年排放量超5000吨的工业企业和市级以上污水处理厂的主体"}}

>>> input：
"战略投资者指符合下列条件之一的机构：①管理资产超百亿 ②有产业整合经验 ③持股锁定期不少于36个月"
<<< output：
{{"战略投资者": "符合管理资产超百亿/有产业整合经验/持股锁定期不少于36个月条件之一的机构"}}

【输出规范】
使用Python列表包裹你找到的所有全局定义，每个定义为一个字典：
<DEFINITIONS>
[
    {{"定义术语1": "包含原文关键限定词的定义描述1"}},
    ...
]
</DEFINITIONS>

下面是需要你处理的法律条文：
{Law_articles}
"""

In [10]:
# content, reasoning_content, api_usage = call_gpt(
#     prompt=prompt_extract_definitions.format(Law_articles=law_articles),
#     api_key = "35684824-1776-48b6-94fd-96c2e99d0724",
#     base_url = "https://ark.cn-beijing.volces.com/api/v3",
#     model = "ep-20250217153824-9xcbx",  # DeepSeek-R1
# )
# print(reasoning_content)

In [11]:
# print(content)

## 2.1 异步获取全局定义

In [ ]:
import pandas as pd
import asyncio
import os


async def get_global_definition_response_async(
    file_name: str,
    batch_size: int = 20,
    max_concurrency: int = 5  # 最大并发数
):
    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir = r"law_to_MEU/st_1_law_csv"
    output_dir = r"law_to_MEU/st_2_law_keywords_definitions/raw_response"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    df = pd.read_csv(input_path, encoding='utf-8-sig')
    law_articles = df['law_article'].tolist()

    batches = [law_articles[i:i+batch_size] 
              for i in range(0, len(law_articles), batch_size)]
    
    tasks = []
    for batch in batches:
        law_text = "\n".join(batch)
        prompt = prompt_extract_definitions.format(Law_articles=law_text)
        task = call_gpt_async(
            prompt=prompt,
            api_key="35684824-1776-48b6-94fd-96c2e99d0724",
            base_url="https://ark.cn-beijing.volces.com/api/v3",
            model="ep-20250217153824-9xcbx",
            temperature=0.6
        )
        tasks.append(task)

    # 分组执行控制并发
    all_results = []
    for i in range(0, len(tasks), max_concurrency):
        batch_tasks = tasks[i:i + max_concurrency]
        batch_results = await asyncio.gather(*batch_tasks)
        all_results.extend(batch_results)

    contents = []
    reasonings = []
    usages = []
    
    for content, reasoning, usage in all_results:
        contents.append(content)
        reasonings.append(reasoning)
        usages.append(usage)

    combined_df = pd.DataFrame({
        'Content': contents,
        'Reasoning': reasonings,
        'Usage': usages
    })
    
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    return contents, reasonings, usages

In [13]:

# file_name = "北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv"
# await get_global_definition_response_async(file_name)


## 2.2 解析并保存全局定义

In [14]:
import pandas as pd
import re
import ast
import os

def get_global_definition(input_file):

    if input_file.endswith(".csv"):
        pass
    else:
        input_file += ".csv"
        
    # 构建路径
    input_dir = "law_to_MEU/st_2_law_keywords_definitions/raw_response"
    input_path = os.path.join(input_dir,  input_file)
    output_dir = "law_to_MEU/st_2_law_keywords_definitions"
    output_path = os.path.join(output_dir, input_file)
    
    # 读取原始文件
    df = pd.read_csv(input_path, encoding='utf-8-sig')
    
    all_key_value_pairs = []
    
    for content in df['Content']:
        # 修复闭合标签格式
        content = content.replace("<\DEFINITIONS>", "</DEFINITIONS>")
        
        # 提取DEFINITIONS块
        match = re.search(r'<DEFINITIONS>(.*?)</DEFINITIONS>', content, re.DOTALL)
        if match:
            definitions_str = match.group(1).strip()
            try:
                # 将字符串解析为列表
                definitions_list = ast.literal_eval(definitions_str)
                if isinstance(definitions_list, list):
                    for item in definitions_list:
                        if isinstance(item, dict) and len(item) == 1:
                            # 提取唯一键值对
                            key, value = next(iter(item.items()))
                            all_key_value_pairs.append({"key": key, "value": value})
            except (SyntaxError, ValueError) as e:
                print(f"解析失败：{e}\n内容：{definitions_str}")
                continue
    
    # 去重逻辑（以key为唯一标识，保留首次出现的值）
    unique_pairs = {}
    for pair in all_key_value_pairs:
        key = pair["key"]
        if key not in unique_pairs:
            unique_pairs[key] = pair
    
    # 构建DataFrame
    unique_df = pd.DataFrame(list(unique_pairs.values()))
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存结果
    unique_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    return unique_df

In [15]:
# file_name = "北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv"
# result_df = get_global_definition(file_name)
# result_df

In [16]:
# len(result_df)

# 3. 提取身份信息转换

## 3.1 prompt模板

## 3.2 异步获取身份转换信息

## 3.3 解析并保存身份转换信息

# n. 整体处理

In [17]:
file_name = "北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv"

In [18]:
# await get_keywords_response_async(file_name)
# result_df_keywords = get_keywords(file_name)
# result_df_keywords

In [19]:
import os

# 获取当前工作路径
current_path = os.getcwd()

# 打印结果
print("当前工作路径:", current_path)

当前工作路径: /Users/svenli/Nutstore Files/0ProjectsOnNut/ForProfYuan/合规的形式化/meu_graph_v2


In [20]:
directory_path = "law_to_MEU/st_1_law_csv"

# 提取所有.csv文件的文件名，不包含路径
law_filenames = [
    f for f in os.listdir(directory_path) if f.endswith('.csv')
]

law_filenames

['北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv',
 '北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv',
 '北京证券交易所上市公司持续监管指引第5号——要约收购.csv',
 '北京证券交易所上市公司持续监管指引第2号——季度报告.csv',
 '北京证券交易所上市公司持续监管指引第1号——独立董事.csv',
 '北京证券交易所上市公司持续监管指引第9号——募集资金管理.csv',
 '北京证券交易所上市公司持续监管指引第4号——股份回购.csv',
 '北京证券交易所上市公司持续监管指引第10号——权益分派.csv',
 '北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.csv',
 '北京证券交易所上市公司持续监管指引第7号——转板.csv']

In [21]:
from tqdm import tqdm

for file_name in tqdm(law_filenames):
    await get_global_definition_response_async(file_name)
    result_df_global_definition = get_global_definition(file_name)
    # result_df_global_definition

100%|██████████| 10/10 [06:09<00:00, 36.96s/it]
